# **Setup**
## Recompile Cython Extensions

In [ ]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

## Imports

In [1]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# System functions
from pathlib import Path

# NLTK resources
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Ngram acquisition functions
from ngram_acquire.pipeline.orchestrate import download_and_ingest_to_rocksdb
from ngram_acquire.pipeline.logger import setup_logger

# Ngram processing functions
from ngram_filter.config import PipelineConfig, FilterConfig
from ngram_filter.pipeline.orchestrator import build_processed_db
from common_db.api import open_db
from utilities.peek import *

In [ ]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

# **Download 5-Grams and Ingest to a RocksDB Database**

In [ ]:
download_and_ingest_to_rocksdb(
    ngram_size = 5,
    repo_release_id = "20200217",
    repo_corpus_id = "eng",
    db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db",
    file_range = (0, 19422),
    random_seed = 11,
    workers = 40,
    use_threads = False,
    ngram_type = "tagged",
    overwrite = True,
    write_batch_size = 100_000,
    open_type = "write:packed24",
    post_compact = True
)

# **Run Processing Pipeline**

In [ ]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db")
dst_db = src_db.parent / "5grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"
wht_path ="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams_processed.db/whitelist.txt"

pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    readers=2,
    ingestors=16,
    partitioning_sample_rate=0.001,
    prefix_length=6,
    mode="resume",
    force_cache_use=False,
    enable_ingest=True,
    delete_after_ingest=True,
    enable_compact=False,
    progress_every_s=60.0,
)

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
    whitelist_path=wht_path
)

build_processed_db(pipeline_config, filter_config)

As the pipeline runs, the work-unit status is tracked in a SQLite database. Run the following command to check progress.

Execute from the `processing_tmp` directory:
```
python3 -c "
import sqlite3
conn = sqlite3.connect('work_tracker.db')
cur = conn.cursor()
results = cur.execute('SELECT status, COUNT(*) FROM work_units GROUP BY status').fetchall()
status_dict = dict(results)
print(f\"Completed: {status_dict.get('completed', 0)}, Processing: {status_dict.get('processing', 0)}, Pending: {status_dict.get('pending', 0)}\")
"
```

## Run a Manual Compaction

Compaction improves DB read performance. Run this if compaction was disabled during ingestion.

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

with open_db(db_path, mode="rw", profile="read:packed24") as db:
    db.compact_all()

# **Inspect the Procesed Database**
## `db_head`: Print the first _N_ key–value pairs

In [27]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_head(db_path, key_format="utf-8", value_format="packed", n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   <UNK> <UNK> <UNK> <UNK> aaa
     Value: [166 records] ... +156 earlier, (2010, 467, 399), (2011, 412, 326), (2012, 360, 297)
            (2013, 266, 223), (2014, 309, 256), (2015, 220, 190), (2016, 232, 204)
            (2017, 193, 183), (2018, 132, 107), (2019, 139, 115)

[ 2] Key:   <UNK> <UNK> <UNK> <UNK> aac
     Value: [122 records] ... +112 earlier, (2010, 119, 98), (2011, 124, 90), (2012, 133, 109)
            (2013, 175, 84), (2014, 168, 126), (2015, 56, 51), (2016, 93, 83), (2017, 118, 71)
            (2018, 112, 71), (2019, 26, 24)

[ 3] Key:   <UNK> <UNK> <UNK> <UNK> aachen
     Value: [143 records] ... +133 earlier, (2010, 216, 152), (2011, 235, 173), (2012, 1900, 1363)
            (2013, 1181, 944), (2014, 223, 169), (2015, 132, 120), (2016, 207, 162)
            (2017, 257, 141), (2018, 192, 140), (2019, 144, 121)

[ 4] Key:   <UNK> <UN

## `db_peek`: Print _N_ key-value pairs starting at the specified key

In [48]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek(db_path, start_key=b"quick brown <UNK> <UNK> <UNK>", key_format="utf-8", value_format="packed", n=5)


5 key-value pairs starting from 717569636b2062726f776e203c554e4b3e203c554e4b3e203c554e4b3e:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   quick brown <UNK> <UNK> <UNK>
     Value: [21 records] ... +11 earlier, (2006, 4, 3), (2007, 1, 1), (2008, 13, 3), (2009, 2, 2)
            (2010, 5, 4), (2011, 2, 2), (2012, 9, 7), (2013, 5, 3), (2014, 2, 1), (2016, 1, 1)

[ 2] Key:   quick brown eye <UNK> <UNK>
     Value: [156 records] ... +146 earlier, (2010, 5, 5), (2011, 6, 6), (2012, 10, 10), (2013, 19, 19)
            (2014, 18, 18), (2015, 22, 22), (2016, 18, 18), (2017, 26, 26), (2018, 99, 99)
            (2019, 16, 16)

[ 3] Key:   quick brown eye <UNK> butler
     Value: [10 records] (1866, 23, 23), (1867, 2, 2), (1869, 2, 2), (1870, 2, 2), (1871, 4, 4), (1875, 3, 3)
            (1891, 1, 1), (1892, 1, 1), (1903, 1, 1), (1908, 1, 1)

[ 4] Key:   quick brown eye take <UNK>
     Value: [36 records] ... +26 earlier, (2007, 1,

## `db_peek_prefix`: Print key-value pairs containing the specified prefix

In [47]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek_prefix(db_path, prefix=b"<UNK> united state <UNK> america", key_format="utf-8", value_format="summary", n=1)

1 key-value pairs with prefix 3c554e4b3e20756e69746564207374617465203c554e4b3e20616d6572696361:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   <UNK> united state <UNK> america
     Value: Total: 20,980,406 occurrences in 6,505,676 volumes (1475-2019, 380 years)

